In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def tfidf_similarity_function (data,column_name):
    vect = TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
            encoding='utf-8', input='content',
            lowercase=True, max_df=1.0, max_features=None, min_df=0.0,
            ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
            stop_words=None, strip_accents=None, sublinear_tf=False,
            token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
            vocabulary=None)
    vect = TfidfVectorizer()                                                                                                                                                                                                   
    tfidf = vect.fit_transform(data[column_name].values.astype('U'))                                                                                                                                                                                                                       
    pairwise_similarity = tfidf * tfidf.T
    return pairwise_similarity.toarray()

In [3]:
def select_related_repos(repo_matrix):
    temp = pd.DataFrame(data=repo_matrix.index.tolist(), columns= ['full_name'])
    temp2 = pd.merge(temp, repo_full_names, how="left", on=['full_name'])
    selected = pd.merge(temp2, df_comments, how="left", on=['repo_id'])
    return selected

In [4]:
file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/All_Developer_Metrics.csv"
df = pd.read_csv(file, low_memory=False)
pd.set_option('display.max_columns', None)
#pd.set_option('display.float_format', lambda x: '%.3f' % x)
#pd.set_option('display.max_rows', None)
df.head()

,repo_id,user_id,num_commit_committed,num_commit_authored,num_commit_commented,num_issue_opened,num_issue_closed,num_issue_assigned,num_issue_commented,num_issue_closedwithPR,num_pr_opened,num_pr_merged,num_pr_assigned,num_pr_commented,count,O_num_commit_committed,O_num_commit_authored,O_num_commit_commented,O_num_issue_opened,O_num_issue_closed,O_num_issue_assigned,O_num_issue_closedPR,O_num_issue_commented,O_num_pr_opened,O_num_pr_assigned,O_num_pr_merged,O_num_pr_commented,isin_top_lang_owned,isin_top_lang_committed,isin_top_lang_watched,code_contributions,comments,commit_2_comment,issue_2_comment,pr_2_comment,code_2_comment,comment_2_code,issue_related,pr_related,commit_related,binary_count,binary_code_contributions,binary_comments,binary_issue_related,binary_pr_related,binary_commit_related
0,1,21,NaN,NaN,NaN,2.0,NaN,NaN,17.0,1.0,1.0,NaN,NaN,NaN,21.0,NaN,NaN,NaN,0.010695,NaN,NaN,0.053476,0.034623,0.009804,NaN,NaN,NaN,1.0,1.0,1.0,4.0,17.0,NaN,0.117647,NaN,NaN,NaN,20.0,1.0,0.0,4.0,3.0,1.0,3.0,1.0,0.0
1,1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,173,NaN,NaN,NaN,2.0,NaN,NaN,136.0,2.0,2.0,1.0,NaN,1.0,144.0,NaN,NaN,NaN,0.010695,NaN,NaN,0.106952,0.276986,0.019608,NaN,0.098039,0.05,1.0,1.0,1.0,7.0,137.0,NaN,0.014706,2.0,2.014706,0.49635,140.0,4.0,0.0,6.0,4.0,2.0,3.0,3.0,0.0


In [5]:
repos = pd.read_csv('/Users/kadir/github-data/Repos.csv',sep=',')
repo_full_names = repos[['repo_id', 'full_name']].copy()
watched_repos = pd.read_csv('/Users/kadir/github-data/Watchers-fullname.csv',sep=',')
forked_repos = pd.read_csv('/Users/kadir/github-data/Forks-fullname.csv',sep=',')
watched_repos = pd.concat([watched_repos,forked_repos]).drop_duplicates().reset_index(drop=True)
watched_repos.set_index('full_name',inplace=True)

df_comments = pd.read_csv('/Users/kadir/jupyter-notebook/TextClassification/comments_topwords.csv',
                    low_memory=False,na_filter=False,na_values=0)


In [6]:
header = 'ALL_developer_metrics'

topn= [1,3,5,10,20]
for n in topn:
    topn = n
    topn_hit = []
    names = []
    for i in range(2,46): 
        name = df.columns[i]
        x = df.loc[:,['repo_id','user_id']]
        y = df.iloc[:, [i]]
        ratings = pd.concat([x, y], axis=1, sort=False)
        ratings.rename(columns={name:'rating'}, inplace=True)  
        ratings.replace(0,np.NaN,inplace=True)
        df2 = pd.merge(ratings, repo_full_names, on='repo_id')
        repo_matrix = df2.pivot_table(index='full_name', columns='user_id', values='rating')
        repo_matrix.replace(np.nan, 0, inplace=True)
        selected = select_related_repos(repo_matrix)
        repo_similarity_matrix = tfidf_similarity_function(selected,'topWords')
        #repo_similarity_matrix = cosine_similarity(repo_matrix)
        repo_matrix2 = repo_matrix.mask(repo_matrix==0, repo_similarity_matrix@repo_matrix.values)
        accuracy = 0
        for id in repo_matrix.columns:
            x = repo_matrix[[id]][repo_matrix[[id]].values == 0].index
            recommends = repo_matrix2[[id]].loc[x].sort_values(by=[id], ascending = 0)
            topn_recom = recommends.head(topn)
            watched = watched_repos[watched_repos.user_id == id]
            # less than (n) change algorithm to calculate topn accuracy
            if len(watched)>topn:
                hit=0
                for n in topn_recom.index:
                    if n in watched.index: #full name matches
                        hit = hit +1
                    else: # only owners or repo name matches
                        wat_owners = watched.index.str.split('/').str[0].to_list()
                        wat_reponame = watched.index.str.split('/').str[1].to_list()
                        rec_owner = n.split('/')[0]
                        if rec_owner in wat_owners:
                            hit = hit + 0.5
                        if rec_owner in wat_reponame:
                            hit = hit + 0.5

                accuracy += hit * 100 / topn
            else:
                hit=0
                for n in topn_recom.index:
                    if n in watched.index: #full name matches
                        hit = hit +1
                    else: # only owners matches
                        wat_owners = watched.index.str.split('/').str[0].to_list()
                        wat_reponame = watched.index.str.split('/').str[1].to_list()
                        rec_owner = n.split('/')[0]
                        if rec_owner in wat_owners:
                            hit = hit + 0.5
                        if rec_owner in wat_reponame:
                            hit = hit + 0.5

                accuracy += hit * 100 / len(watched)

        total_accuracy = accuracy / 100
        topn_hit.append(total_accuracy)
        names.append(name)
        display(name + ':'+ str(total_accuracy))
    topn_hit = np.round(topn_hit, 2)
    hit_text = "hit_top_"+str(topn)
    results  = pd.DataFrame({hit_text:topn_hit,'metric':names})
    filename = r'/Users/kadir/Desktop/comment_similarity_scores_'+header+'_'+hit_text+'.csv'
    export_csv = results.to_csv (filename,index=False, header=True)

'num_commit_committed:29.0'

'num_commit_authored:28.5'

'num_commit_commented:26.0'

'num_issue_opened:26.0'

'num_issue_closed:20.0'

...

'binary_code_contributions:16.5'

'binary_comments:15.4'

'binary_issue_related:14.975'

'binary_pr_related:18.275'

'binary_commit_related:17.925'

In [7]:
def highlight_max(data, color='yellow'):

    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [8]:
file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_1.csv"
dftop1 = pd.read_csv(file)
dftop1.sort_values(by=['hit_top_1'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_3.csv"
dftop3 = pd.read_csv(file)
dftop3.sort_values(by=['hit_top_3'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_5.csv"
dftop5 = pd.read_csv(file)
dftop5.sort_values(by=['hit_top_5'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_10.csv"
dftop10 = pd.read_csv(file)
dftop10.sort_values(by=['hit_top_10'], ascending = 0, inplace=True)

file = "/Users/kadir/jupyter-notebook/DeveloperMetrics/data/comment_similarity_scores_ALL_developer_metrics_hit_top_20.csv"
dftop20 = pd.read_csv(file)
dftop20.sort_values(by=['hit_top_20'], ascending = 0, inplace=True)

result = pd.merge(dftop1, dftop3, how="left", on="metric")
result = pd.merge(result, dftop5, how="left", on="metric")
result = pd.merge(result, dftop10, how="left", on="metric")
result = pd.merge(result, dftop20, how="left", on="metric")

# WE ADD THIS SCORES MANUALLY.... '!!!!!!'

#add to sun's papers metric scores. (from features_of_ref10)
#new_row = {'metric':'Sun\'s metric', 
#           'hit_top_1':8.00, 'hit_top_3':6.60, 'hit_top_5':7.10, 'hit_top_10':7.35, 'hit_top_20':6.5}
#result = result.append(new_row, ignore_index=True)

#add mean of topk scores to each metric
result['mean'] = result.mean(axis=1)
result.set_index('metric', inplace=True)
result.style.apply(highlight_max)

,hit_top_1,hit_top_3,hit_top_5,hit_top_10,hit_top_20,mean
metric,,,,,,
num_pr_opened,35.500000,29.500000,24.700000,22.300000,18.950000,26.190000
num_issue_closedwithPR,34.000000,29.830000,24.200000,21.700000,18.020000,25.550000
binary_commit_related,31.000000,25.000000,23.000000,20.200000,17.920000,23.424000
num_commit_committed,29.000000,24.500000,22.600000,18.500000,15.280000,21.976000
binary_pr_related,28.500000,30.500000,27.500000,21.900000,18.270000,25.334000
num_commit_authored,28.500000,24.500000,23.100000,18.450000,15.350000,21.980000
num_pr_merged,28.000000,21.330000,20.700000,18.900000,17.050000,21.196000
O_num_commit_authored,28.000000,22.170000,20.100000,17.700000,15.120000,20.618000
code_2_comment,27.500000,22.170000,22.200000,19.400000,17.120000,21.678000


In [ ]:
comment_similarity_